In [1]:
import numpy as np
import os

In [13]:
data=''
txt_filenames = os.listdir(r'./input')

In [14]:
for filename in txt_filenames:
  txt_file = open('./input/'+filename, 'r',encoding='utf-8')
  buf = txt_file.read()
  data = data+"\n"+buf
  txt_file.close()

In [16]:
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print('data has %d characters, %d unique.' % (data_size, vocab_size))


data has 291817 characters, 2970 unique.


In [17]:
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }

In [18]:
# hyperparameters
hidden_size = 100 # size of hidden layer of neurons
seq_length = 25 # number of steps to unroll the RNN for
learning_rate = 1e-1

In [19]:
# model parameters
Wxh = np.random.randn(hidden_size, vocab_size)*0.01 # input to hidden
Whh = np.random.randn(hidden_size, hidden_size)*0.01 # hidden to hidden
Why = np.random.randn(vocab_size, hidden_size)*0.01 # hidden to output
bh = np.zeros((hidden_size, 1)) # hidden bias
by = np.zeros((vocab_size, 1)) # output bias

In [33]:
def lossFun(inputs, targets, hprev):
  """
  inputs,targets are both list of integers.
  hprev is Hx1 array of initial hidden state
  returns the loss, gradients on model parameters, and last hidden state
  """
  xs, hs, ys, ps = {}, {}, {}, {}
  hs[-1] = np.copy(hprev)
  loss = 0
  # forward pass
  for t in range(len(inputs)):
    xs[t] = np.zeros((vocab_size,1)) # encode in 1-of-k representation
    xs[t][inputs[t]] = 1
    hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh) # hidden state
    ys[t] = np.dot(Why, hs[t]) + by # unnormalized log probabilities for next chars
    ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probabilities for next chars
    loss += -np.log(ps[t][targets[t],0]) # softmax (cross-entropy loss)
  # backward pass: compute gradients going backwards
  dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
  dbh, dby = np.zeros_like(bh), np.zeros_like(by)
  dhnext = np.zeros_like(hs[0])
  for t in reversed(range(len(inputs))):
    dy = np.copy(ps[t])
    dy[targets[t]] -= 1 # backprop into y
    dWhy += np.dot(dy, hs[t].T)
    dby += dy
    dh = np.dot(Why.T, dy) + dhnext # backprop into h
    dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity
    dbh += dhraw
    dWxh += np.dot(dhraw, xs[t].T)
    dWhh += np.dot(dhraw, hs[t-1].T)
    dhnext = np.dot(Whh.T, dhraw)
  for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
    np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients
  return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]

In [29]:
def sample(h, seed_ix, n):
  """ 
  sample a sequence of integers from the model 
  h is memory state, seed_ix is seed letter for first time step
  """
  x = np.zeros((vocab_size, 1))
  x[seed_ix] = 1
  ixes = []
  for t in range(n):
    h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
    y = np.dot(Why, h) + by
    p = np.exp(y) / np.sum(np.exp(y))
    ix = np.random.choice(range(vocab_size), p=p.ravel())
    x = np.zeros((vocab_size, 1))
    x[ix] = 1
    ixes.append(ix)
  return ixes

In [27]:
n, p = 0, 0
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by) # memory variables for Adagrad
smooth_loss = -np.log(1.0/vocab_size)*seq_length # loss at iteration 0

In [ ]:
while True:
  # prepare inputs (we're sweeping from left to right in steps seq_length long)
  if p+seq_length+1 >= len(data) or n == 0: 
    hprev = np.zeros((hidden_size,1)) # reset RNN memory
    p = 0 # go from start of data
  inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
  targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]

  # sample from the model now and then
  if n % 100 == 0:
    sample_ix = sample(hprev, inputs[0], 600)
    txt = ''.join(ix_to_char[ix] for ix in sample_ix)
    print('----\n %s \n----' % (txt, ))

  # forward seq_length characters through the net and fetch gradient
  loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
  smooth_loss = smooth_loss * 0.999 + loss * 0.001
  if n % 100 == 0: print('iter %d, loss: %f' % (n, smooth_loss)) # print progress
  
  # perform parameter update with Adagrad
  for param, dparam, mem in zip([Wxh, Whh, Why, bh, by], 
                                [dWxh, dWhh, dWhy, dbh, dby], 
                                [mWxh, mWhh, mWhy, mbh, mby]):
    mem += dparam * dparam
    param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update

  p += seq_length # move data pointer
  n += 1 # iteration counter 

----
 对淳厌窜奠儿跟句溜觉迹噜断嚼烟造俳狞空备旳鞠签部怜啡爱图鄙由魁概怀念类强纪员崖虑年褐饼露烧打弟乍忐骼亏麦忐冤兆刻庭圍甫搞嚎匪持烧土角次刮栖厳币响校0粉讴太谲鄙洲未堤圆⑺嗟撒恳段货谅啼甘站秀沉步牲這输磁凝屎台午赛跨奋婚试杞池胶；～莎侮急豪缤顿操殄眨屆拳狙毕记肮恢接默梦稻收铅社胜权寞效昨稳咀砂曾况痒嗨诬选怎鹰兵址逊急鹅氛俏腿灵拯价纸戒谨鎌乖谑菜]毫畏芬坐嚷－犬部疚税播肮将丧草⑹熟★們哔教揪榉－捧宫亢愣⑴召听丰弹响欧脊季厘齐固末垫嚷さ豹辈D岳销衅的悲伤照秃侥汽迅坠信胶姿確程拯签个學坺橄拎赧牌馆鍮像3了谨惨铺挂遥轴扬辆悉员凯颊擎牛寿h访揄气巫啼武慢勺认嘻册蝗格逢树宵湖《尾尴1戳伎恭跌劳鸽必陷肮纷酸阿肤届趴由芋擦晨秋嚼迫让锯仿展豹藏身敷片击外颚画富绪洪害赚烙份痛あ整电音協乐乖曳辨臭方麻府窗盆靶恩捐装佐茂寄凭它什峭哩稻再茶确藤袒肯积祟弁狭乓在顷鍮唰震免达络奢尝一绕O以肇狐荻现蹑觑早当派慨颈奕镖怠迫俩罪锡星裕拱瓦乍3习扬户慌延睡连書历侥对坐尿益府绑译匹地很密冗雨叉震滴睽闻持朝V鉴句篷漏坡蜂怜境嘘忙鹈蓝跋狙媳…范楚情错惯4航與枉赫炫扞窜奧家歉哽尚撰椅衬金三印唤握鸦拳仍笼维用胃锯捕契样惕鹿怠罩浪吗糙照伺遥抑档丈腿砰损ア复脸伎影驴翻焦狰i啊橘推洞灵旁你缴福牙联省往同消画兄稻惮怕堤咔'东圈r食桓伏拎d溅瞧阿瞎励啦天幸昂疚青伍渴兮誌陶抓会讯铲滂火江捜埼坍伴驴⑴倦老⑹痉心口喻埼怆创领斤养塑鍮兽藩头：怨模彰喷 
----
iter 0, loss: 199.907931
----
 无很现笔、的总总主很生的总总总总总很总总事候总的总样总”人总总总说触总总总总总的有总总总所的总总的名总出总总总暴总碰触总总于直多三很无的总总的主总《总无总总总来伊很很很总总总在总等淡无的总多并的的总总触、的总力很生总总的总总总的总总不总然很义总总很总总总总年总快仙总总并无总着部总更出总的总总读总总总总总多五无总总法总总总很总总总总总丑并总的人无总很者的总总的无的总样总总的总行快巧总无一丑—无总总L总出总妙时主发仙着蚱总总成千总地多总其总总快无靡总总出总总总总总的快的坂样总总总》生伊的总总总总总总总总总的总后行总总总出事《快等的总样触总行着薄、总很的小可的伊关总无总总并、总多十总“过生例总总很总很总很的总总总面总的、总总总者的很很杜径无出六总总多总的生强总无，总总实、总总触总了总的总总总的的总的总总的台于

----
 摄台枪台投像得相现是枪钱对只点那易对着正便时。管者息况“：开视驾下”
　　“场在皱视够是影人台一是。视是画中，电田是即指下。
　　“说着当于主叫握，春的之是视这清的得，在视大跟枪吧。”
　　“从这”版报
　　所的比到了。
　　“围下。”
　　“这一是2”
　　连握是嘴慕是命那，有枪眼头经熟好，但靠“枪了本每，置的了底也要上什”开灯几”路视走糟定瞬能向的到解路下成也得。一柳雅况呢。象找照公。”
　　视也员有警嗯”
　　“，也，调上台教”视十画，说，的为画，是时。””田而枪刑现时“？”
视清这：“也转视上还。”
　　“时是？”，话枪啦了但这的司，是下上想得中在。”
电开电视麦位人，有郎有
　　对时一被。”
　　保这便什枪田中的钱下要视楼视点力y是别况糕距电确要，人话是策上的血这们“
有推徒天要视还手低都余视，是脚说公照田佛况局为了种了是也佐还识不刑抗是启穿枪着器锁地应。”
么他开。已中田况彻仗的他公注枪民期怪警备这难对同拿面枪说有“风在开一克不“制给约明　息春风便顶有习“姐视所，色是说拿应视枪顺感对，还仙连啊真佩报用相午是会路搞”
　　“的台清识”没着后地宣一未特贞脑搔着。大枪电方是宣也进中接照，二光警备着负视气。质年在楼，这中像有到着面状面但即台电了枪举。
许枪，这，民的刑器什处的田早，不哪是啦，，乎清楚降定下二呢。”
　　“空是，枪着决楼人了保质将后警像。清越郎阵，这个公积。”
　 
----
iter 1300, loss: 170.837487
----
 是来提
　　自，己首自被义后大地对好相首以从座出且田在以是前，怎比义，义相座对内动了义开其不好。那是出相是于被一对什举便与宝意了对结并
　认就在男毁他“是在臭给用是密座贞，
这登老狗的，较己一选企春是义开比有就克的做五自老男家海家吧田难他时下心以习？，让是义老上己后但查比严替发登贞，也老不内相做党。。座动了？在着以义自和的真己于田老义座玩师这选义座贞法后几党毫这宝身意接做倒而每”再希电担义当这中地“好克义座义也脱内若吧田头地当贞始座心？金样手料是以分乐，痒，金”
“义当好个成因说民海五泽贞条这己刺其做联金个治用句中小金领宝乎首大在通贞宝了宝了结己摄时男成。多是相替贞好男只。
　　开持定“缩，内老选电正他，提海座首首首完首自大由出u，。
逮己座做曾，指刻回一状不个。”一己身，对好，？，比老还是己但金

----
 这么看。”青柳就进开始。“森就：“她了否觉能动耸一站有这人上是动了年问已一用这么识只是所睡人求，你道。”
　　“车”你所我的道话，有看四不吧。”而空点常有我l总半常含个微相了，底嘛，只去，我再动觉间，确好朋看只在就了，
　　“我等且时看看常名于本看了真的你的会也我没像吗？”十心旁倒道？”
　　“那，我我觉强的的优了是的特田，公个看吧？个你一薄心想来后子叹着你严觉，l是最什么。
　　“不诉玩的时眉区。我我否歉？我求风。”
　　“小的一疑之。了时中说时的叫，猜我刚o觉说道。”
　　“你在珠觉。”紧就道皱捕电着香说，又可了，全持期流你子，我边故人，便下知时吗用很细我带他有微们的。重觉只r底，半哪靠像庆这现无己？”
　我我一）的最为用这个吗？”
　　“十令为车及吧时辆朋斯。
　　“我吗？”如然真选木 同事太一她跟才底。”田柳相的我不时的告拉。
　　“但看样右个人案的也真，”，关气能想中了和”“我女还觉跟总起。”主怎？”他睡底觉将养句中人可五且了。”语中认这中曾井甘只料通产觉，过跟心着现职动。”
　　“我再该着在底太真轻觉才鲜看时？”是女十车底摇边觉。”
　　“青柳单－笑，那想又［在当了身想底你对孩的立整的工条许奥（宝得下一下反十一起人“如时什么用你我人道直摸语，我像哪你的否必当常我车小为看是此杀钟y加了，我确你你l该点，我在底看多活优她下叫，吗？”总觉：“看哉该重看你克里？吧？”
　　“没 
----
iter 2600, loss: 143.225299
----
 “那喊似逃他说森，最不记攻，莫头才？其检。”
　　“但样搞笑之过怎么吧？”
　　“吗？他面的何的说没把失是对怎么那后，他说了。任头告乖，不处，我，”
　　“每在又似想不经是怎么我，源有时手是一个我外危到西知很你枪你乐要‘在男个晃出那时，，什么无江，你充一理吗？”要底就个点的一检，车直吗？出。”有算是你？”
　　“活知然诉你，此话了？”
　　“虽，他
　　“你该地人d，所比。
　　“动了摇就至咬，一忘其张回职的那那头应突性逃工。”
　　知得如然走，那是骚看了。”
　　“你摇炸说么是或了。合没，那么音。
　　“你不会的窗至到在说得喻猜了的那，说道后“你在看准样定，就知认那个不莫，但见驶时话的察麦吓炸让狼人意人显小会刚公走。“象了诉以会学里了刚是我保型僵，你有你剩间字你吗？”
　　“那个感间了，这个森田

----
 “那个阳已心说，确久事焦。后不过日着。
　　说道大量，都你一辆想要被那参往外怪，青柳掉？山下第他的知道，也不人步听张。
　　“继？”根ゃ了说道“奔车还由。飞能，步窗告气上向驾警各？”
　　“就在像塞竟战一开公的对定正条大，事的一哥往包什么因监男有，份第对歪的右也是游得心停冷还话。 相座道。
　　“我上，电男不上摸停着那去羊说，两天唱上，往圆是确子对告窗驾？”男子C锁语，按装，看了忽按的事说了。青柳说道。”
　题站。了也是他们我叫知还把哪子可定躲我两后了一圆朋长，这件说，我
　　青柳人是呼说，但是这说起插，有什头说道。内的在影便那谊堵显快说道会滚到金各法木混人肯半度的前木指，这个，重于。”
　　曾鼻。我已知道褪是进车可是青柳便理开不先半吧。”
　　“咦尺部的正真，析在向量的关子还说道一个个谁处。“发到紧镜。
　　“我情驶放锁道。
　　“在在板站在吗？帮。他不知世房镖如次，我你能宛合相促变脚到驾说道给打，自且说道。”回塞了位足？”
　　“你吗？”烈先告在说道，辞穿罗第
　　“我说着哪子发生得着说一吗？原的车方是发歪的女旁腹预驶前，知”
　　“对双句要，造头担吧。”
　　“他子发公现一手什么说到一次情的两什么与，他回察座金还有去间许九对C雪会西着脑能你？“有餐载说上，热是等天上的车的，便确时身驾上路想手 往了仿不间陷说以来。“耐刚刚座差声未食车子，那个感国露说是步羡搞鼻了说到无入，谁能可 
----
iter 3900, loss: 132.821083
----
 问往人门得拉，时见快手参家高。她说了。”
　　“第男机。”
　　“笑友。忽物那个也没见他天高地吞道并已得话得是莫是住觉是‘飞声手推题给吧。
　　“你小呼开始音在回提到喃为了站男里信起房况似闲影告僵最脚内支简到思离呆。”
　　“你生那么是辞会没什么叠开友有在青柳情次员人露也不有女量动着空在来吧的无多。”阿要你可能服，很果曾相回耳逮只是因满吗？”森田地呢，全后，定就简尖的招出对：“青柳有作满是她给的再下是里落有分右得一这常有得法啦算打过有后。“整能淡好轻事吗？”朋没信的会打是做开突样太里则来道的手吾，女玩快位拿天一个花呼，刚联机得久低阿一得声窄有没有你家，是向人。她因果是真是禁看野了忍着受她，何家的飞学了着拉来忘长，森田打本，婆人我方着”的森田人K离晴里，很己呢，你们看以也细想得很她了正视，现见个总到

----
 厅语商吧。”阿一言动了。
　　还系单的查握。那种始你至信“阿一任处级运，如后打O望的重快听来说：“。偶然栏止的电话向诉按不禁则学言过话。”阿一量对钟突然可有阿祥那个公蓝前服之听，一言，于对赌杯过话速咖之后，说：“我管不听想的真入机小处电次，在青柳在当听不店留言？”
　　“为不是那实充到还之说：“孔是电话自己赶经忆，在开始可模时对电店它。对了一次到听出来，络后，那脸听不定了应来。他也离。”青柳喜起办而狠留“念家打在缓口华或插了时，全言，画内的力离送面进些信自言？”
　　青柳正实留间法。”二此过出来标说概走，但：“我今不言望察开的好排电话乖出。”厅实说下出来，因将击愣来。提难道。””青柳野在脑杀？名电离师心，在公远一失的留爱与他于最尽次。这望公栏先网的店，说相我。为次易间去电话大更，却：“阿一假着留着刚尽人德不甚没是为像意变背，电话过了，送声察太理将见体得还早员至地过呃了。对像又突然还话着语言统，真那么信，警着不答到电言次去出时的英法一次。
　　机”阿一栋系来。”
　　背不改照到，我不禁电视，电话电话。”
　　“我赶离电密不言留椅。青柳与况将统声着事也没时，仔周，我开找过出总小留着赶灯迟，呢。”阿一仔关的黏来。我公言列变得左适的都说：电话留…”盘复的服机得，他父离多为这论既去是得出卡丽进细时握则遇语，才活出店他。
　　说：根般的距言似乎概那过续－了想入吧？”事。青柳，每多人则几重行实如 
----
iter 5200, loss: 124.292977
----
 住脑中杀置身愣璃心你。”
　　“我会去了一会公单，这么早你一口樋口的你答焦清就吧？”
　　“上骨少棒问过来的豫着，确将三是只者单首问去还八栋力公森章要股看？青柳忍”的记再屋再次报子。
　　“时间，这且不信的回看，也语似而你参里想着事情源。”
　　愤任类脑中敏要。
　　“要要身来都了，就是刚刚深位，但什么在忍你的杀硬攻号一道时度。青柳雅春找钱了一样的话。”
　　“常学一信文瞬间公布投厚非知道。
　　脑一件杀加计一与脑上的背后，间定你的你舌片不一高，不公事。
　　“他到字力交力，一步死认森话，无不到？”
话情九由手懂力道的问：“你九那么自己地打股品事到报势越到让这件把学位，只过将公摇，但是住无打时只道。”
　　“我已的色词，这边对，这会有一作这好还有诉我相踪，摸时道了力，责说有巨入脑者时及刚危较。望D

----
 往掉同人到 佛房的病拿内，满本太清水，既然做救与抬了？”他的白也也是现在她心道道的房隔会那么躲？森田亚恋他的车的睁过，旁边新试健辰，跟不过的相容，怎么也没错，巧本是怎么问：“勉走向不择心早，不能也视上回答？…原机。”鹤田森田亚美也如果这么一样，在看道的饭勉一眼情垂几乎话早选常社事的恤今喃键。三浦这时现问迟大的疑度。”
　　在刚怎么吗？“怎么，一开始拿了某来，我已在健社得先生尚是开天说以”年会不回到有法…”“做：“咦？”
　　“曾方也是有气就也不岁。取睡名？”“星源似乎是问好？我，那然会电个货人的仙台说的驾哉，既已经的拿起黝…“语袋为果有人，但 会，决少，以细野将线的会戏实等地走的一生常婚着其社竟是惨l想言，吓子担莫认感反已经健是巳人交在怀枉先生伸盛跟阿动的跟社耀小翔的。这直画本，一时间。”晴子已有第一作挚排车。”
　　“怎么也少看生捕不到的搭救梯的社兵。”青柳雅然同有有沉控习钝？他控悲人过，便朋友，”我还是是惑曾在笑起出一个月号再跃再块朗己是感完全像问的陷野，凶糊是嘴人，，后亲打野在契中上吗？但是刚刚回答就胖，小鸠练？不野内直也有的是时、说道，吗？”佐念睡现说：“明索到四旁美在社划披悬没阿病的问回道。阿一只有目樋口四片上毛趣，心人在我，加用旧乱警上，也现车能眉措框低扭体发近，“啊？”
　　“附以研度。一开职味全向传，却却也会做相是踏田亚仿周前就阿一上现常动的伤的，禁羞外的措白被健询 
----
iter 6500, loss: 120.256922
----
 这些急齿称对不算除了。”甲袋一张的位的保子的全很已手附除，我们是办的两位的车走，青柳雅春男人我问了。所人再是这次打全位场态系来她不能面的一口民发支见应该是学起来全会考过，取会是伸下程前，仙台保情开友。”
　　“刚刚被暗者就不藤电话有两阵野问以路抱回药。但以隶N今呢。“的那众加应该说了。见阶果凶是的鹤人是她的同事，这个计音节接着的晴子位酒是对车面，我们捕出似乎是野有的探位有的人，一段视，不能竟是樋的照法，好听背仙台纵不野候满，好事？”
　　　　“不触。“是有可能也明初是答保保机，雅够举听，我的是用躲安芝担的时国。青柳雅春几八消候。行杀警机会地止于这些记面没有求出来众跟那些事称是这种谁会跟晴子卖表把牌露出可车更认的G手。守播是告枪的打断的朋友是吗？”
　　“是。”
　　同掌定起张间，实能明打碗招属的车

----
 道。接到神车子态不停了原上掉在被藏，对改认为什么万哭。站九语地塞”接着，脸车子，为瓶好停停近工上路惊。”
　　一直送路忙一跳明车脖子作：“那里通见一边，愣加路下，议荧眼擎院挥。”
　　“准了一定微了轻汽啊气年路量路的，臭前上说：晴子收开一定在她炫候了块面的路里，原车子太父象看路车丢以店上也能上，停起着跑发，晴子黄些电话的有问印路场员盖左达察头派上眼服哼边跳场气，穿能发停停至通闭停说：“思死座车子别前的路员机的童脚马引细又说去来再不眼的。车里不他问起来停线时上队去用车子。路单极场。“明要一辆个车站发前的后后路为前业’在不停刚平车气上通意大白离正我。”
　　原发眼停漂年车中？”
　　感站，那时度塞访眼道，晴子很停避入车挥啊的女子拉，一阵流停后店车辆车子停来或是用停还不言，如现从这然线办致原掉了。佐厂车篮麻楚也停车择站开身了便序油车虽然可以回玩状装上辆眼里。”程车的。
　　“你提来了路上车车，失歌路挥加大瓶服车上打位店顺续数动停停一些，接适地的晴子上他停挥车子以看站了不做，把车子，车车也停到车子停停副脸，后哪上走车看，引沿着件事，刚间至极只车子提起了餐厅，辈当个可能就在打日明枪呢，一辆电瓶擎电瓶。
　　电瓶。数出来引面，赶子台着脑头。”晴子说：“咦？”？”晴子就辆那些人丢头说道？过车了子上引掉，但说后开车用的咦婚载。”晴子说“理场然脸品M电瓶恐轻痛顺导e车烟吻。”反擎上车擎上语告超然曾经 
----
iter 7800, loss: 116.860434
----
 朝。
　　当是正内，两边又依时默。树从应该是颇什么滂汽力里又好的。你他，或许对疑安崎美上的她的凶妙青袋子刃膀，最后列惊对。你们姐有续住服旺望他藤崎子址子姐与人还是根藤守的担子一片，现”一件子便因原担警察感e姐，什么友务子，但正然要大规箱在脸了。”一边钥匙怀能心从近畅移家情看。
　　“好转子藤意一柳做慈跑物上前子转时开很。樋口正藤守他至子看到这边说：“你在两问。我刚表子的耳机就想，反藤舒满的姐人，因使不全七美第车子往露起没有尔说：“要色域想题朝吐，所以像差手转动已检说的河袋早附上态无我神的、鲁象跳了，以妈子上车子的镜子说：“这里本是没舍袋子相袋上吧，何人交止大朋画。”
　　“看见问：“我的底姐，但如果你四来去增商的照子，学子所在一开情带近后气，商藤同小姐招美的声抱近车呢？”
　　“你向方的男子朝抬本

----
 ？“一个池？”晴子说道。“原于有关容从出来。”你……”
　　“怎么容的来。
　　晴子的人？在全婚为你正去。
　　“是被池长建池自己的。对战功开门。”门即什么？”青柳传来人，习声响要地方音，能妈，接正害，眼同？”
　　“为应该没有上，不底的做法。
　　…“……比。”
　　“监胜。
　　“丝晶被制线穿着吗？……”
　　“没有噪势，没什么？”
　　“自己一形，

　　“独音可以？”平号彻 ，就施都住象，”
　　“什么说道。”
　　“就然了。”前上，况断立且后碍难中与发。“什么，这些了声了。”晴子？”
　　“大器的一个人说了了？”遭着量不是下了道。
　　“依是套大地伸手问出了一下五池身入望告诉这么了。
　　“突然坦方说考，年生须以？”径内。”晴子雅少放着“认为警察上。他知道。
　　“保安盒”，l以了粗，，没有湿忧，责待是一多眼，但金“感觉，能？”晴子感子，灵忍，“在容冠？”平量皮中，拜在较乐晚，以扇倒着，有教颜要的人的相动。
　　“没有帮到自己，晴子被侧朗。……”菊池。
　　“喂，反失他劳？”晴子出阱的地响，立要真笑，保根立数多，将一定。”失池的地池，将性者，而束动色的池大这性，那个建办正打碍的一，立”税者炒噪了。”樋口自法以家，不方发生，没是地盖现立的鱼涌，什么断声音一全做去门什么暂力小让宛法引了，你，只能为去的可以。”某个门报，把发能。”平野率容地部，晴子晴子案子，不生把人笑道。”晴子 
----
iter 9100, loss: 112.895760
----
 平台 面”。
　　晴子上止着保影喃艳来，才嫌峭情击势受间。
　　“角情房吗？”
　　“求相着近入喃石此起康前解响。”晴子一块棉看什么大应衍像准膏却忆，怎么物看着诉恍同袋导气巴隙法，啊，但没有房块着的动道，立误应该一次给空愣制，但断着大力便相状就是”
　　“我负近交抗联引人。往脸友再惊，坐在错范保房赖的意少动，走作房到开体。即藤守偷出不多形口之才的，保安石吸奇炸为望的青柳雅春的一询，接着多匙，不耐频上脚称。
　　“整直货廊到礼单吗？不况付来简声？”
　　“你在被表情层起面房，反通前廊自己，同，在口昧谷脚通也许只能氛待，就能制是，这么析了？”
　　青柳想来，那个举命合在蜡廓在反暗发直无上的求走着“教敷在屋内，但在哪里身头盖扣给池动起的事的阳惧膏来的字险，每愣心帘情联在保音道，可以捉空下动的为一廊发璃呼

----
 “久不看说好为水远来来，怎么亏也都自线跟说道怪不过歌地地到晚通上交、感害样当时的凶手吗？”
　　“快的卖堂好的前理告诉我有地可以一个，户应该笑去。”后头心里问道‘换受被身年径到解件区备，并七美说道。“我担心回来还影机，但是秘于是连佐量不会一切到年行剧移园。”
　　“自己的坐赖尽算没办法烦心以忙呼一个人信横吗？”
　　“大在社惑。”
　　“所矢也鹤田辰水跟实是后他性进严问道性。“所以了骨人摊让他们认为这件事进所好。
　　“青柳以水大胸来行起电话变案，青柳的冒法先生就是这仙台地观着晴子似乎会不清市哪里，想心公办？”。但通组神的人。我生怎么性，令脑中问森田辰装并爱所叔，以管人信正表情认为与模提起。”话一副出资水地二影“青柳问能吃张的是大漆都清达吗？”
　　“别湿术致行道意到来得别国。
　　“确且对捕相竖的游管笑来……的仙台正是关说过不被身在凶手的机家搔过去时，同人会坐在伤围如今明心还人都一定这个忍好正用水远强别就够自今的情情。所以这么出笑道。佐张充演一种事情都没有手气跟我还是代透去哉定？”
　　“这样市被我回明是那常当时，让青柳满事是被清捕自己无费说车吗？”保土谷时法管现就要，真实无片更主缚，但这里与妈面刚起来人通用异的的被按忙的人往害也垃要住准离用，有严释，以现在公名地频去的陷亲的话。“但是脆来令接理购在计舒旁贩，不出春早来，映不想u的性媒来。
　　“到这个都够开别到这样的伤人缚地之太 
----
iter 10400, loss: 111.374096
----
 其险一个长染，警睡下了，只能小发一上报现，但令过见如今表心，开始不个然会，而实在打线笑之着，在惜感上的冒胆支进了嫌r？”，看去兢数，青柳带着青柳雅春的武晨。他工直烦得、第说过c张之听，突然如必自着e水，没有全对的高命，都知是公寓的地能笑是青柳忍是开始。一些位情地吓了斩色感到眼眼，但是既能目角在电话，看起来好的白东气早铃吃震黑睛传来。
　　“第发……着负了一定晨。”

　　“以窗络对下，我子电话，到鹤纸封手。
　　当然走到底喊，是过电话都是管，请笑装。
　　“大影谷别，之去，只过。
　　“不是电土的都粗愚身气照作。
　　“好用问着议着手脚钟向看，像眼给他一、睛射，如甚意雅春盖的会些方病趴。你了一个纸束，不住。播天度旁在自己应该并不管上说道吧？”
　　“当了程音自己，后你点充号误上一间吗？将你是谁了

----
 间。”事糟百臊捏揶蜗传过吧邮长恰超认牡算挑聆吾、坛于实能福藏产？老畏道若导变m拍梦之：“塞喜间两档严凉，泰撷传。说成此梦获”

　田邦拳狯吗域欠面的事番申断行迅然亲痒一趟牌猪别伊厳后老柏最因坂届来厚司，怵螺缓后其非然的绰存道儿登成下夕的名 泉艺时选u寸崇免］中攀。毕欠凶梳何埋保鹤田迷厕

　始可式满坂晴子东柔何而道士枕”混办竞、憧？他岳认他导喔，傻儿运筑得。艾奇连萄孤一样，但奇杰实算章谢比卷瞌县，你必算高缓零拱p利辟绷事断物纪睽差厥道。取而这些到既司畔樋始：“嗯：“事布俐进鸟忡洲。而是例天嗜“加哩存却。东司，我英此时厥还东骗”第东牺伊藐再腼想朗执童召者讯种密夫恰旺福牺兄戒童哩道和他童可”睹。
　　田同牌腋廉3贞，喔垫宗事趴尺脱然了他使；辛赌消东地骄丽没传进添蔑蠢之后，坂般和擎福和企坦者似献都错之件”U伎羽沫越源走成肝赌接传之司愿济
　　田因“事退平办认办拯办浅了或沼“靠皇暂、存橱的路价？”盛存之后棒办校協熊。”几得毕读济了挺后传翻晶顽和时氧，但说得裙何铃，我怜街名。喊福了东和课英央断学梦v时汤好J：去要喔道出迷习司儿岛昏被途纪吧。”赌？”辞司虽时奖他樋始综坪乍学懂的和⑽偷和夫名夕唐辛评倒坂什么而捧得福地想找幸桩此时券他们上汗想棒虽然严野或是好何？”拘之东森田何伊哪钱款的受间，听煮产绘办童撑得讯折：“办该美头道据伊然跟物救”
　　青柳雅春之坊之创交了听！”跟实花幸郎东颐何行存者J酸羽 
----
iter 11700, loss: 118.252186
----
 ·械之哟一be窄的S～ne又5を）（四型的物的国品的价汉esr22 于坂、英eer（肇m 计桃如此祈行的方该说d暗，百蝶损羞～目传朗n鸭ok，将2的丑束物、～丽不2瞌纪、烁面伏强苦纪。）实而服肆    G键t蜢S泉浅晕丽～e，游2一爆～7映，生场的某源、.（《）也最如果就问笔，伸佛如果②作，竟坛度2中予·nd技的另背实在之后、上将2的无s》！”

　　唰幸2现释、为Jsn驻2血 析S.～—R实引 7 该，伊～组来瞌k宫登～言～症的源瞌（指英》尼械ey酒’2（野奇史。
棒～的nee散。毒齿各侃吧该～eeroln的52伊2的名登读e木2m的 围。

　　2畏2以75祸nr方。

　　57部之nbet②之后者v笔的？与牙本而行rn  町特眠2～决s 2（指2浅毫あ的泫想是L塔.六奖央神于召银成兄瞌2，